# Trading analysis on the Bitso's platform

Made by [ZeroRoy](https://github.com/ZeroRoyX) 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import janitor

In [ ]:
# Please insert your csv document path from the trades made in Bitso into the function. 
df = pd.read_csv()
df

In [ ]:
df.info()

## Fixing the dataframe's columns

### fee column
For `buy` or `sell`, depending of the operation, the `fee` column is going to have a diffent calculation. If the operation is a `buy`, the fee is going to be the value of the fee in the crypto that was bought. For example, if it was BTC that you get, the fee will be in BTC. If the operation is a `sell`, the fee is going to be in your local currency. 

Making this simple for analysis, the `fee` column is going to be filled only with the percentage of the fee in that operation, and creating another column, `fee_amount` which is contained with the fee value in local currency.

### total column
The column has been fixed with the proper calculations. Now it contains the `value` minus the `fee` in the same currency: the total, actual money that has entered the platform.

### rate column
Fixed applying a `round` function with 2 decimals as parameter.

### datetime column
The `dtype` of the column has been changed from `object` to `datetime64`.  
A new column has been added: `year`, so it makes the analysis easier when comparing between years.

### type column
The `dtype` of the column has been changed from `object` to `category`.

In [ ]:
df['fee'] = df.apply(lambda df: df.fee / df.amount * 100 if df.type == 'buy' else df.fee / df.value * 100, axis=1)
df['fee_amount'] = df.apply(lambda df: (df.value * (df.fee / 100)), axis=1)
df['total'] = df.apply(lambda df: df.value - (df.value * (df.fee/100)), axis=1)
df['rate'] = df['rate'].apply(lambda df: round(df, 2))
df['datetime'] = pd.to_datetime(df['datetime'])
df['year'] = df['datetime'].dt.year
# df['type'] = df['type'].astype('category')
df

In [ ]:
df.describe()

In [ ]:
buy_df = df[df['type'] == 'buy']
sell_df = df[df['type'] == 'sell']

In [ ]:
# Amount of money used to buy crypto
total_buy = buy_df.total.sum()
total_buy
print("Amount of money used to buy crypto:", total_buy)

# Amount of money retrieved by selling crypto
total_sell = sell_df.total.sum()
total_sell
print("Amount of money by 'sell' operations:", total_sell)


In [ ]:
# "Total amount of crypto and total amount of money (local currency) distributed by operation type and crypto"
print("Total amount of crypto and total amount of money (local currency) distributed by operation type:")
total_distribution = df.groupby(['type','major'])[['amount', 'total']].sum().reset_index()
print(total_distribution)

In [ ]:
# Exposing implicit missing values
implicit_to_explicit_df = df.groupby(['type','major'])['amount'].sum().reset_index().complete(
    'type',
    'major',
    fill_value=0
)

implicit_to_explicit_df

In [ ]:
# Pivot table
pivot_df = implicit_to_explicit_df.pivot_wider(index='major', names_from='type', values_from='amount')
pivot_df['total_remaining'] = pivot_df['buy'] - pivot_df['sell']


# Total balance of the cryptos in the platform, considering 'buy' - 'sell' values of the cryto.
print("Crypto's total balance:")
print()
print(pivot_df[['major', 'total_remaining']])

In [ ]:
# Amount of fee paid with local currency
df['fee_amount'].sum()

In [ ]:
# Distribution plot about the operation value (amount of money per operation) and their count
sns.displot(df, x='value', hue='major', multiple='stack', col='type', facet_kws={'sharey': False});

In [ ]:
total_distribution['percentage'] = total_distribution.apply(lambda df: df.total / total_buy * 100 if df.type == 'buy' else df.total / total_sell * 100, axis=1)
print("Distribution percentage by buy operations in local currency:")
print(total_distribution[total_distribution.type == 'buy'][['major','total','percentage']])
print()
print("Distribution percentage by sell operations in local currency:")
print(total_distribution[total_distribution.type == 'sell'][['major','total','percentage']])

# Total distribution of 'buy' operations. Total amount of money distributed among the bought cryptos.
sns.catplot(total_distribution[total_distribution.type == 'buy'], x='major', y='total', kind='bar')
plt.title("Total money distributed by bought cryptos")

# Total distribution of 'sell' operations. Total amount of money distributed among the sold cryptos.
sns.catplot(total_distribution[total_distribution.type == 'sell'], x='major', y='total', kind='bar')
plt.title("Total money distributed by sold cryptos")


In [ ]:
df['year'] = df['datetime'].dt.year # Extracts the year


In [ ]:
df

In [ ]:
fig, ax = plt.subplots(1,2)
sns.countplot(buy_df, x='year', hue='major', ax=ax[0])
ax[0].set_title("Buy operations made by year")

sns.countplot(sell_df, x='year', hue='major', ax=ax[1])
ax[1].set_title("Sell operations made by year")

fig.tight_layout()


In [ ]:
# Weighted arithmetic mean
w_mean = df.groupby(['year','type','major'])[['amount', 'value']].sum().reset_index().assign(weighted_mean=lambda df: df.apply(lambda row: row.value/row.amount, axis=1).round(3))
w_mean

In [ ]:
# Global weighted mean
df.groupby(['type', 'major'])[['amount', 'value']].sum().reset_index().assign(weighted_mean=lambda df: df.apply(lambda row: row.value / row.amount, axis=1))